### IMPORTINGS

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import mlflow
from mlflow.models.signature import infer_signature

In [2]:
df = pd.read_csv("data/IMDB Dataset.csv", nrows=300)

## Exploratory data analysis

In [3]:
df.head()

review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     300 non-null    object
 1   sentiment  300 non-null    object
dtypes: object(2)
memory usage: 4.8+ KB


In [5]:
df["sentiment"].value_counts()

sentiment
negative    161
positive    139
Name: count, dtype: int64

# I. Using Mlflow

### 1. Preprocessing

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import nltk
import mlflow
from mlflow.models.signature import infer_signature
from scipy.sparse import csr_matrix 

nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and punctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Tokenization
    tokens = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the preprocessed tokens
    preprocessed_text = ' '.join(tokens)

    # Apply CountVectorizer to the preprocessed text
    vectorizer = CountVectorizer()
    preprocessed_text_vectorized = vectorizer.fit_transform([preprocessed_text])

    # Convert the sparse matrix to a dense array
    preprocessed_text_array = preprocessed_text_vectorized.toarray()

    return preprocessed_text_array

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/olivier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/olivier/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
df['vectorized_text'] = df['review'].apply(preprocess_text)

In [38]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['vectorized_text'], df['sentiment'], test_size=0.2, random_state=42)

In [39]:
from sklearn.preprocessing import LabelEncoder
# Encodez les étiquettes textuelles en valeurs numériques
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [43]:
X_train

232    [[1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,...
59     [[1, 1, 1, 2, 1, 5, 1, 1, 1, 1, 1, 1, 2, 1, 1,...
6      [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
185    [[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 4, 2,...
173    [[1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 5, 2, 1, 1, 1,...
                             ...                        
188    [[1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,...
71     [[1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,...
106    [[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,...
270    [[2, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 3, 1, 1, 1,...
102    [[2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 1,...
Name: vectorized_text, Length: 240, dtype: object

### 2. ML models

In [42]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Set up MLflow experiment
mlflow.set_experiment("sentiment_analysis")

# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Train and log metrics for each model
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # Log metrics
        mlflow.log_metric("accuracy", accuracy)

        # Log the model
        mlflow.sklearn.log_model(model, model_name)


ValueError: setting an array element with a sequence.

In [21]:
!mlflow ui 

[2023-11-27 10:44:35 +0100] [66587] [INFO] Starting gunicorn 21.2.0
[2023-11-27 10:44:35 +0100] [66587] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-11-27 10:44:35 +0100] [66587] [ERROR] Retrying in 1 second.
[2023-11-27 10:44:36 +0100] [66587] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-11-27 10:44:36 +0100] [66587] [ERROR] Retrying in 1 second.
[2023-11-27 10:44:37 +0100] [66587] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-11-27 10:44:37 +0100] [66587] [ERROR] Retrying in 1 second.
[2023-11-27 10:44:38 +0100] [66587] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-11-27 10:44:38 +0100] [66587] [ERROR] Retrying in 1 second.
[2023-11-27 10:44:39 +0100] [66587] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-11-27 10:44:39 +0100] [66587] [ERROR] Retrying in 1 second.
[2023-11-27 10:44:40 +0100] [66587] [ERROR] Can't connect to ('127.0.0.1', 5000)
Running the mlflow server failed. Please see the logs above for details.


In [13]:
loaded_model = mlflow.sklearn.load_model("runs:/6e212333915c4a0a8b248cd46eb57c32/Logistic Regression")

In [14]:
# Convert into SVC model into ONNX format file
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 1]))]
onx = convert_sklearn(loaded_model, initial_types=initial_type)
with open("outputs/log_reg_model.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import mlflow
import onnx
import pickle
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType
from skl2onnx.common.data_types import FloatTensorType

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

df['processed_review'] = df['review'].apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(df['processed_review'], df['sentiment'], test_size=0.2, random_state=42)
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

models = {
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

mlflow.set_experiment("polarity dectection")

for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        model.fit(X_train_vectorized, y_train_encoded)
        y_pred = model.predict(X_test_vectorized)
        accuracy = accuracy_score(y_test_encoded, y_pred)
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("accuracy", accuracy)
        mlflow.log_params(model.get_params())

        # Log the model
        mlflow.sklearn.log_model(model, model_name)

best_model = mlflow.sklearn.load_model(f"runs:/59d5cda72be94f21b84f9d72b72aa039/Gradient Boosting")

# Save the preprocessing transformations in pickle format
with open("outputs/vectorizer.pkl", "wb") as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)
with open("outputs/label_encoder.pkl", "wb") as label_encoder_file:
    pickle.dump(label_encoder, label_encoder_file)

# Convert the best model to ONNX format
initial_type = [('float_input', FloatTensorType([None]))] 
onx = convert_sklearn(best_model, initial_types=initial_type)
with open("outputs/best_model.onnx", "wb") as f:
    f.write(onx.SerializeToString())

NameError: name 'df' is not defined